# <h1 align='center'>  Amazon question/answer Classification. </h1>

### OBJECTIF :

L'objectif de ce projet est de réaliser des classifications automatiques des données de questions-réponses issues de la base de données Amazon afin d'examiner la pertinence des groupes obtenus. En d'autres termes, l'objectif est de regrouper les questions-réponses en fonction de leurs caractéristiques communes, en utilisant des techniques d'apprentissage automatique, et d'évaluer la qualité de ces groupes formés.

### Data Description :

Ces ensembles de données contiennent des questions et des réponses sur les produits de l'ensemble de données Amazon ci-dessus.

- **Questions :** L'ensemble de données comprend environ 1,48 million de questions.
- **Réponses :** Il y a environ 4 019 744 réponses correspondant aux questions.
- **Questions avec oui/non étiquetées :** Sur l'ensemble des questions, environ 309 419 sont étiquetées comme des questions binaires (oui/non).
- **Nombre de produits uniques avec des questions :** L'ensemble de données comprend des questions relatives à 191 185 produits uniques.
- **Métadonnées :** L'ensemble de données inclut les métadonnées suivantes pour chaque question et réponse :
- **Texte de la question et de la réponse :** Le texte réel de la question et de sa réponse correspondante.
- **Étiquette de question binaire :** Indique si la question est binaire (oui/non) ou non, et si oui, si elle a une réponse oui/non.
- **Horodatage :** Informations relatives au temps, comme la date de la question et de la réponse.
- **ID du produit :** Ce champ fournit une référence au produit correspondant dans l'ensemble de données des évaluations, permettant ainsi une analyse ultérieure ou une intégration avec d'autres ensembles de données d'Amazon.
Ces informations nous donnent un aperçu de la structure de l'ensemble de données et du nombre de questions, réponses, produits uniques et questions binaires étiquetées qu'il contient.


### Data Processing :

La fonction **parse(path)** utilisée pour lire un fichier compressé au format GZIP et extraire les données ligne par ligne. 

In [1]:
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield eval(l)

Le code utilise la fonction **parse(path)** pour lire un fichier compressé GZIP contenant des données de questions-réponses d'Amazon au format JSON, puis écrit ces données dans un fichier de sortie au format JSON.

In [5]:
import json
import gzip

def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

f = open("output.strict", 'w')
for l in parse("/content/drive/MyDrive/qa_Beauty.json.gz"):
  f.write(l + '\n')

Le code utilise la fonction **parse(path)** pour lire un fichier compressé GZIP contenant des données de questions-réponses d'Amazon, puis convertit ces données en un objet DataFrame de pandas.

In [6]:
import gzip
import pandas as pd

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('/content/drive/MyDrive/qa_Beauty.json.gz')

In [7]:
df

,questionType,asin,answerTime,question,answerType,answer,unixTime
0,yes/no,602260074X,10 days ago,can you fit make up brushes in the trays,Y,"yes it comes with adjustable dividers, you can...",NaN
1,yes/no,602260074X,"Mar 3, 2015",Can you move all the dividers?,?,"yes,all the provided dividers are adjustable",1.425370e+09
2,yes/no,602260074X,"Dec 30, 2014",is the surface in side the smooth?,Y,Yes,1.419926e+09
3,open-ended,602260074X,"Jul 13, 2014",How deep do the extending trays measure?,NaN,"Hi there, Not too deep. Maybe like an inch dee...",1.405235e+09
4,yes/no,602260074X,"May 21, 2014",Can bottles of nail polish stand upright in th...,?,No. We just tried it and it won't.,1.400656e+09
...,...,...,...,...,...,...,...
42417,yes/no,B00L5JHZJO,"Jul 2, 2014",I find myself with rough cuticles right around...,Y,"Yes, you can. In the evening before you go to ...",1.404284e+09
42418,yes/no,B00L5JHZJO,"Jul 2, 2014",is it good for nail beauty?,?,I would say it's good for cuticles. I can't sa...,1.404284e+09
42419,open-ended,B00L5JHZJO,"Jul 2, 2014",how can i use it for Topical Use on Dry Hair?,NaN,"A little goes a long way! A drop or two, depen...",1.404284e+09
42420,open-ended,B00L5JHZJO,"Jul 2, 2014",how can i use it for In-Shower Application?,NaN,Application during shower takes less time. Aft...,1.404284e+09


- 'questionType': Il s'agit du type de question posée. Dans cet exemple, il est défini comme 'yes/no', ce qui signifie que la question peut être répondu par oui ou non.

- 'asin': C'est un identifiant unique pour le produit auquel la question et la réponse se réfèrent. Il est utilisé pour identifier de manière unique le produit dans la base de données.

- 'answerTime': Il indique le moment où la réponse a été donnée. Dans le premier exemple, il est spécifié comme '10 days ago', ce qui signifie que la réponse a été donnée il y a 10 jours à partir du moment où les données ont été collectées.

- 'unixTime': C'est une autre représentation temporelle de la réponse, donnée sous forme de nombre entier représentant le nombre de secondes écoulées depuis le 1er janvier 1970 à minuit UTC.

- 'question': Il s'agit de la question posée par l'utilisateur concernant le produit.

- 'answerType': Cela représente le type de réponse donnée à la question. Dans le premier exemple, il est défini comme 'Y', ce qui peut signifier que la réponse est positive ou affirmative.

- 'answer': C'est la réponse à la question posée par l'utilisateur.

### Spark Session :

utilise PySpark pour créer une session Spark.

In [9]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=f28c03d4934768debf2cca8688f343a3e5a6fac0b5aa75d1a5bae219baf2a5b1
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [10]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()


**spark = SparkSession.builder.getOrCreate()** crée une instance de SparkSession. **SparkSession** est l'entrée principale pour l'utilisation des fonctionnalités de Spark SQL. La méthode **builder** est utilisée pour configurer la session Spark, et **getOrCreate(**) permet de récupérer une session existante ou d'en créer une nouvelle si aucune session n'existe.

### Spark DataFrame :

convertit le DataFrame pandas (df) en un DataFrame Spark, puis effectue des opérations Spark sur ce DataFrame.

In [11]:
# Convert the pandas DataFrame to a Spark DataFrame
spark_df = spark.createDataFrame(df)

# Perform Spark operations on the converted DataFrame
# For example, you can show the first few rows
spark_df.show()


+------------+----------+------------+--------------------+----------+--------------------+-----------+
|questionType|      asin|  answerTime|            question|answerType|              answer|   unixTime|
+------------+----------+------------+--------------------+----------+--------------------+-----------+
|      yes/no|602260074X| 10 days ago|can you fit make ...|         Y|yes it comes with...|        NaN|
|      yes/no|602260074X| Mar 3, 2015|Can you move all ...|         ?|yes,all the provi...|1.4253696E9|
|      yes/no|602260074X|Dec 30, 2014|is the surface in...|         Y|                 Yes|1.4199264E9|
|  open-ended|602260074X|Jul 13, 2014|How deep do the e...|       NaN|Hi there, Not too...|1.4052348E9|
|      yes/no|602260074X|May 21, 2014|Can bottles of na...|         ?|No. We just tried...|1.4006556E9|
|  open-ended|602260074X|Feb 25, 2014|what are the Weight?|       NaN|Light box. Carry ...|1.3933152E9|
|  open-ended|602260074X| Dec 3, 2013|What are the dime...|     

La sortie affichée montre les premières lignes du DataFrame Spark spark_df avec des colonnes telles que questionType, asin, answerTime, question, answer, et unixTime. Chaque ligne correspond à une question-réponse d'Amazon.

### Data Cleaning :

Le code utilise des fonctions de Spark SQL pour compter le nombre de valeurs manquantes (NaN) dans chaque colonne du DataFrame Spark spark_df et affiche ces comptages.

In [12]:
from pyspark.sql.functions import col, isnan, sum as spark_sum

# Iterate over each column and count the NaN values
nan_counts = [(column, spark_df.select(spark_sum(isnan(col(column)).cast("integer"))).collect()[0][0]) for column in spark_df.columns]

# Print the NaN counts for each column
for column, count in nan_counts:
    print(f"Column '{column}': {count} NaN values")


Column 'questionType': 0 NaN values
Column 'asin': 0 NaN values
Column 'answerTime': 0 NaN values
Column 'question': 0 NaN values
Column 'answerType': 24331 NaN values
Column 'answer': 0 NaN values
Column 'unixTime': 1632 NaN values


En exécutant ce code, nous obtiendrez l'affichage des comptages de valeurs NaN pour chaque colonne du DataFrame Spark spark_df. Cela peut nous aider à identifier les colonnes avec des valeurs manquantes dans nos données de questions-réponses d'Amazon.

### Data Transformation :

Le code définit deux dictionnaires : **contractions** et **special_characters_dict**. Ces dictionnaires sont utilisés pour standardiser et rendre le texte plus formel et cohérent dans le jeu de données.

In [14]:
#  Removing contractions to standardize the text and make it more formal and consistent throughout the dataset.
contractions = { 
    "ain't": "am not",
    "aren't": "are not",
    "can't": "can not",
    "can't've": "can not have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
    }

# Removing special characters helps to standardize the text by eliminating non-alphanumeric characters.     
special_characters_dict = {
    "%": " percentage ",
    "€": " euro ",
    "&": " ampersand ",
    '₹':' rupee ',
    '@':' at ' ,
    '$':' dollar '
}


# # Converting special characters to their textual representations helps to enhance the semantic clarity of the text. 

special_number={
    ',000,000,000 ': 'b ',
    ',000,000 ': 'm ',
    ',000 ': 'k '  
}


### Traitement des donnees

Nous avons téléchargé les ressources nécessaires de **NLTK**. La ressource **'punkt'** fournit le support de la tokenisation, et la ressource **'wordnet'** donne accès à la base de données lexicale WordNet.

In [15]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Nous avons créé des fonctions de prétraitement du texte qui effectuent plusieurs étapes essentielles pour nettoyer et normaliser nos données. 

In [18]:
import re
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(sentence)
    #'v' spécifie que nous souhaitons lemmatiser les mots comme des verbes. 
    lemmatized_tokens = [lemmatizer.lemmatize(token, 'v') for token in tokens]
    lemmatized_sentence = ' '.join(lemmatized_tokens)
    return lemmatized_sentence

def preprocess(text):
    # Convert text to lowercase and remove leading/trailing spaces
    text = str(text).lower().strip()
    
    # Convert special numerical representations
    text = re.sub(r'([0-9]+)000000000', r'\1b', text)
    text = re.sub(r'([0-9]+)000000', r'\1m', text)
    text = re.sub(r'([0-9]+)000', r'\1k', text)
    
    # Expand contractions
    text_decontracted = []
    for word in text.split():
        if word in contractions:
            word = contractions[word]
        text_decontracted.append(word)
    text = ' '.join(text_decontracted)
    
    # Replace special characters with their textual representations
    text_special_characters_dict = []
    for word in text.split():
        if word in special_characters_dict:
            word = special_characters_dict[word]
        text_special_characters_dict.append(word)
    text = ' '.join(text_special_characters_dict)
    
    # Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # Remove punctuation
    pattern = re.compile('\W')
    text = re.sub(pattern, ' ', text).strip()
    
    # Lemmatize words
    text = lemmatize_sentence(text)
    
    return text

L'ensemble de ces étapes de prétraitement permet de nettoyer et de normaliser les données textuelles, ce qui facilite leur analyse et leur utilisation ultérieure. 

In [19]:
# Example:
preprocess("I've already! wasn't <b>done</b>?")

'i have already be not do'

Le code permet d'appliquer la fonction preprocess en tant que fonction définie par l'utilisateur (UDF) sur les colonnes "question" et "answer" du DataFrame Spark spark_df. La fonction preprocess_udf est créée à l'aide de la fonction udf() du module pyspark.sql.functions.

In [20]:
from pyspark.sql.functions import udf

# Define a UDF from the preprocess function
preprocess_udf = udf(preprocess)

# Apply the UDF to the 'question' and 'answer' columns
spark_df = spark_df.withColumn('question', preprocess_udf(spark_df['question']))
spark_df = spark_df.withColumn('answer', preprocess_udf(spark_df['answer']))


In [21]:
spark_df.show()

+------------+----------+------------+--------------------+----------+--------------------+-----------+
|questionType|      asin|  answerTime|            question|answerType|              answer|   unixTime|
+------------+----------+------------+--------------------+----------+--------------------+-----------+
|      yes/no|602260074X| 10 days ago|can you fit make ...|         Y|yes it come with ...|        NaN|
|      yes/no|602260074X| Mar 3, 2015|can you move all ...|         ?|yes all the provi...|1.4253696E9|
|      yes/no|602260074X|Dec 30, 2014|be the surface in...|         Y|                 yes|1.4199264E9|
|  open-ended|602260074X|Jul 13, 2014|how deep do the e...|       NaN|hi there not too ...|1.4052348E9|
|      yes/no|602260074X|May 21, 2014|can bottle of nai...|         ?|no we just try it...|1.4006556E9|
|  open-ended|602260074X|Feb 25, 2014|  what be the weight|       NaN|light box carry i...|1.3933152E9|
|  open-ended|602260074X| Dec 3, 2013|what be the dimen...|     

Les colonnes "question" et "answer" ont été prétraitées en convertissant le texte en minuscules, en supprimant les espaces en début et en fin de texte, en étendant les contractions, en remplaçant les représentations numériques spéciales, en supprimant les balises HTML, en supprimant la ponctuation et en lemmatisant les mots.

In [22]:
spark_dff= spark_df.drop("answerType","answerTime","unixTime","asin")


Nous avez créé un nouveau DataFrame appelé spark_dff en supprimant les colonnes "answerType", "answerTime", "unixTime" et "asin" du DataFrame spark_df

In [23]:
spark_dff.show()

+------------+--------------------+--------------------+
|questionType|            question|              answer|
+------------+--------------------+--------------------+
|      yes/no|can you fit make ...|yes it come with ...|
|      yes/no|can you move all ...|yes all the provi...|
|      yes/no|be the surface in...|                 yes|
|  open-ended|how deep do the e...|hi there not too ...|
|      yes/no|can bottle of nai...|no we just try it...|
|  open-ended|  what be the weight|light box carry i...|
|  open-ended|what be the dimen...|i be not sure but...|
|  open-ended|what be the tray ...|there be 4 trays ...|
|  open-ended|how can i get a p...|                good|
|  open-ended|i have use the pe...|thank for your qu...|
|  open-ended|goodnight crompre...|i own this unit a...|
|      yes/no|do it work for pe...|no it do not the ...|
|      yes/no|do it come with e...|                 yes|
|  open-ended|how i know if the...|why be you ask it...|
|  open-ended|be this product i

Le DataFrame spark_dff contient maintenant les colonnes "questionType", "question" et "answer" après avoir supprimé les colonnes spécifiées.

### Embedding Creation :

Ce code utilise la classe Word2Vec du module pyspark.ml.feature pour entraîner un modèle Word2Vec sur les données de la colonne "question" du DataFrame split_df. 

In [25]:
from pyspark.ml.feature import Word2Vec
from pyspark.sql.functions import split

# Split the 'question' column into an array of words
split_df = spark_dff.withColumn('question_words', split(spark_dff['question'], ' '))

# Apply Word2Vec to the 'question_words' column
word2vec = Word2Vec(vectorSize=100, inputCol='question_words', outputCol='question_vectors')
word2vec_model = word2vec.fit(split_df)

# Apply the Word2Vec model to transform the 'question_words' column
split_df = word2vec_model.transform(split_df)

Le modèle Word2Vec est appliqué au DataFrame split_df en utilisant la méthode transform, ce qui génère une nouvelle colonne "question_vectors" contenant les vecteurs de mots calculés à partir des mots de la colonne "question_words".

In [26]:
split_df.show()

+------------+--------------------+--------------------+--------------------+--------------------+
|questionType|            question|              answer|      question_words|    question_vectors|
+------------+--------------------+--------------------+--------------------+--------------------+
|      yes/no|can you fit make ...|yes it come with ...|[can, you, fit, m...|[0.02948325955205...|
|      yes/no|can you move all ...|yes all the provi...|[can, you, move, ...|[0.04757946970251...|
|      yes/no|be the surface in...|                 yes|[be, the, surface...|[0.10754237590091...|
|  open-ended|how deep do the e...|hi there not too ...|[how, deep, do, t...|[0.08646665779607...|
|      yes/no|can bottle of nai...|no we just try it...|[can, bottle, of,...|[0.09079277978162...|
|  open-ended|  what be the weight|light box carry i...|[what, be, the, w...|[-0.0132861584424...|
|  open-ended|what be the dimen...|i be not sure but...|[what, be, the, d...|[0.07267820090055...|
|  open-en

Le DataFrame split_df contient maintenant deux nouvelles colonnes : "question_words", qui contient un tableau de mots de la colonne "question", et "question_vectors", qui contient les vecteurs de mots calculés à partir de ces mots à l'aide du modèle Word2Vec entraîné précédemment.

### assembler_df

Le DataFrame assembler_df a été créé en sélectionnant les colonnes pertinentes pour le VectorAssembler :

In [27]:
# Select the relevant columns for the VectorAssembler
assembler_df = split_df.select('question_vectors', 'answer', 'questionType')


Les colonnes sélectionnées sont "question_vectors" (vecteurs de mots de la question), "answer" (réponse) et "questionType" (type de question).

Le code utilise le **VectorAssembler** pour assembler les vecteurs de la colonne "question_vectors" dans une seule colonne de fonction appelée "features". Le nouveau DataFrame est nommé "assembled_df".

In [29]:
# Assemble the vectors into a single feature column
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['question_vectors'], outputCol='features')
assembled_df = assembler.transform(assembler_df)


In [31]:
assembled_df.show()

+--------------------+--------------------+------------+--------------------+
|    question_vectors|              answer|questionType|            features|
+--------------------+--------------------+------------+--------------------+
|[0.02948325955205...|yes it come with ...|      yes/no|[0.02948325955205...|
|[0.04757946970251...|yes all the provi...|      yes/no|[0.04757946970251...|
|[0.10754237590091...|                 yes|      yes/no|[0.10754237590091...|
|[0.08646665779607...|hi there not too ...|  open-ended|[0.08646665779607...|
|[0.09079277978162...|no we just try it...|      yes/no|[0.09079277978162...|
|[-0.0132861584424...|light box carry i...|  open-ended|[-0.0132861584424...|
|[0.07267820090055...|i be not sure but...|  open-ended|[0.07267820090055...|
|[0.03919244249877...|there be 4 trays ...|  open-ended|[0.03919244249877...|
|[-0.0249414243735...|                good|  open-ended|[-0.0249414243735...|
|[0.03593867900781...|thank for your qu...|  open-ended|[0.03593

Le DataFrame résultant contient les colonnes d'origine ainsi que la colonne "features" qui contient les vecteurs assemblés.

### indexed_df

Le code utilise le **StringIndexer** pour indexer la colonne 'questionType' et crée une nouvelle colonne 'label' qui contient les valeurs indexées.

In [33]:
# Index the 'questionType' column
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol='questionType', outputCol='label')
indexed_df = indexer.fit(assembled_df).transform(assembled_df)


In [34]:
indexed_df.show()

+--------------------+--------------------+------------+--------------------+-----+
|    question_vectors|              answer|questionType|            features|label|
+--------------------+--------------------+------------+--------------------+-----+
|[0.02948325955205...|yes it come with ...|      yes/no|[0.02948325955205...|  1.0|
|[0.04757946970251...|yes all the provi...|      yes/no|[0.04757946970251...|  1.0|
|[0.10754237590091...|                 yes|      yes/no|[0.10754237590091...|  1.0|
|[0.08646665779607...|hi there not too ...|  open-ended|[0.08646665779607...|  0.0|
|[0.09079277978162...|no we just try it...|      yes/no|[0.09079277978162...|  1.0|
|[-0.0132861584424...|light box carry i...|  open-ended|[-0.0132861584424...|  0.0|
|[0.07267820090055...|i be not sure but...|  open-ended|[0.07267820090055...|  0.0|
|[0.03919244249877...|there be 4 trays ...|  open-ended|[0.03919244249877...|  0.0|
|[-0.0249414243735...|                good|  open-ended|[-0.0249414243735...

Le DataFrame résultant contient les colonnes d'origine ainsi que la colonne 'label' qui contient les valeurs indexées pour la colonne 'questionType'.

### Data Splitting :

In [35]:
# Split the data into training and testing sets
train_data, test_data = indexed_df.randomSplit([0.8, 0.2], seed=42)

Le code divise les données en ensembles d'entraînement et de test à l'aide de la méthode **randomSplit**. Il utilise un ratio de 80% pour l'ensemble d'entraînement et de 20% pour l'ensemble de test. La graine (seed) est fixée à 42 pour assurer la reproductibilité des résultats.

### Model Training :

Le code utilise un modèle **RandomForestClassifier** pour l'apprentissage. Le modèle est configuré avec les colonnes featuresCol='features' (contenant les caractéristiques des données) et labelCol='label' (contenant les étiquettes des données). Cela signifie que le modèle utilisera les caractéristiques des données pour prédire les étiquettes.

In [37]:
# Train the model
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol='features', labelCol='label')

### Pipeline :

In [39]:
from pyspark.ml import Pipeline
# Create a pipeline
pipeline = Pipeline(stages=[rf])


# Train the model
model = pipeline.fit(train_data)

Dans ce code nous avons créé un pipeline avec une seule étape contenant le modèle RandomForestClassifier. Un pipeline est utilisé pour chaîner différentes étapes de traitement des données et d'apprentissage automatiquement.

### Predictions :

In [40]:
# Make predictions on the test data
predictions = model.transform(test_data)

In [42]:
predictions.show()

+--------------------+--------------------+------------+--------------------+-----+--------------------+--------------------+----------+
|    question_vectors|              answer|questionType|            features|label|       rawPrediction|         probability|prediction|
+--------------------+--------------------+------------+--------------------+-----+--------------------+--------------------+----------+
|[-0.2282457351684...|yes there be a sl...|  open-ended|[-0.2282457351684...|  0.0|[13.0689151309662...|[0.65344575654831...|       0.0|
|[-0.2151029556989...|although the bott...|  open-ended|[-0.2151029556989...|  0.0|[6.69880956969952...|[0.33494047848497...|       1.0|
|[-0.2119010519236...|            no manly|  open-ended|[-0.2119010519236...|  0.0|[12.1343807425312...|[0.60671903712656...|       0.0|
|[-0.1856990994678...|it be eau de colo...|  open-ended|[-0.1856990994678...|  0.0|[14.8017971491586...|[0.74008985745793...|       0.0|
|[-0.1841507721692...|have naturally si..

Le modèle utilise les caractéristiques (features) extraites des données de test pour prédire les étiquettes correspondantes. Les prédictions sont ensuite ajoutées comme colonne "prediction" dans le dataframe "predictions".

### Model Evaluation :

 la précision (accuracy) du modèle sur les données de test.

In [43]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 75.07%


 la précision obtenue est de 75,07%. Cela signifie que le modèle prédit correctement la classe de l'objet dans environ 75,73% des cas sur les données de test.

In [ ]:
predictions.show()